In [1]:
from datetime import date
import os

In [2]:
import sqlite3
from contextlib import closing

# Konfigurieren der Typen Umwandlung

Wir müssen Python sagen wie das Datum genau umgewandelt wird für die benutzte Datenbank. Das können wir in der `data_access/__init__.py` machen, da dies ausgeführt wird, sobald das modul oder eine Komponente aus dem Modul importiert wird.

In der Datei steht zur Konfigurierung folgendes:
```python
from datetime import date, datetime
import sqlite3

## Adapter: Wandelt `date`-Objekt in `TEXT` um
sqlite3.register_adapter(date, lambda d: d.isoformat())

## Konverter: Wandelt gespeicherte `TEXT`-Werte wieder in `date`
sqlite3.register_converter("DATE", lambda s: datetime.strptime(s.decode(), "%Y-%m-%d").date())
```

Deshalb importieren wir hier das modul, das konfiguriert folglich den adapter und converter.

In [3]:
import data_access # import und damit Konfiguration der Umwandlung von date in TEXT und umgekehrt.

# Verbindungsaufbau

Nun müssen wir jedoch im Verbindungsaufbau konfigurieren, dass die definierten Umwandlungen verwendet werden sollen. Das machen wir, indem wir beim Verbinden folgenden Parameter setzen `detect_types=sqlite3.PARSE_DECLTYPES`

In [4]:
def get_connection():
    # Verbindung mit `detect_types=sqlite3.PARSE_DECLTYPES` erstellen
    return sqlite3.connect("example.db", detect_types=sqlite3.PARSE_DECLTYPES)

Auch wenn die Libraries nach dem `with`-Block die Verbindungen oder die Ressourcen freigeben sollten, machen dies nicht alle immer sauber. SQLite3 gehört zu denen die den Context Manager nicht umfänglich implementiert haben.

Python stellt für diese Fälle eine funktion `closing(...)` zur Verfügung, damit die Verbindung durch den context manager `with` auch automatisch geschlossen wird.

Beispiel:
```python
with closing(get_connection()) as conn:
    cursor = conn.cursor()
    ...
```

Damit stellen wir sicher dass die Verbindung und die Ressource sicher wieder freigegeben wird nach dem `with`-Block.

# Erstellen einer einfachen Datenbank
Wir erstellen ein ganz einfaches Datenbankschema mit einer Tabelle:  

__User__ 
* <u>_id_</u>: INTEGER
* name: TEXT
* birthdate: DATE

In [5]:
with closing(get_connection()) as conn:
    try:
        cursor = conn.cursor()
    
        # Falls nötig, Tabelle erstellen
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS User (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                birthdate DATE
            )
        """)
    except sqlite3.Error as e:
        conn.rollback()
    else:
        conn.commit()
    finally:
        cursor.close()

# Hinzufügen eines Beispiel Datensatz

Nun können wir das date von Python verwenden, um ein Datum in das korrekte Format zu konvertieren. Wie oben beschrieben haben wir das durch den Import `import data_access` bereits konfiguriert.

In [6]:
with closing(get_connection()) as conn:
    try:
        cursor = conn.cursor()
        cursor.execute("INSERT INTO User (name, birthdate) VALUES (?, ?)", ("Alice", date.today()))
    except sqlite3.Error as e:
        conn.rollback()
    else:
        conn.commit()
    finally:
        cursor.close()

# Auslesen der Daten
Nun können wir die erstellten Benutzer auslesen und bekommen wieder ein Datum zurück.

In [7]:
with closing(get_connection()) as conn:
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT id, name, birthdate FROM User")
        rows = cursor.fetchall()
        for row in rows:
            print(row)
    except sqlite3.Error as e:
        conn.rollback()
    else:
        conn.commit()
    finally:
        cursor.close()

# Aufräumen
Löschen wir die Datei wieder um aufzuräumen.

In [8]:
from ui.input_helper import input_y_n, YesOrNo

if input_y_n("Wollen Sie die Datei 'example.db' löschen? (y/n) [y]", default=YesOrNo.YES):
    # Überprüfen, ob die Datei existiert, bevor sie gelöscht wird
    if os.path.exists("example.db"):
        os.remove("example.db")
        print("Datei 'example.db' wurde gelöscht.")
    else:
        print("Datei 'example.db' existiert nicht.")
else:
    print("Vorgang abgebrochen.")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a2bdea36-0144-499d-8063-c73fb2a08fa1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>